In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import country_converter as coco
from selenium import webdriver as driver
from geopy.geocoders import Nominatim
import folium

In [ ]:
#scraping data from a website
df = pd.read_html('https://f1.fandom.com/wiki/Circuits')[0]
df = df[df["Season(s)"].str.contains("present") == True]
df = df[df["Location"].str.contains("Miami Gardens, FL")==False]
df = df[df["Location"].str.contains("Zandvoort")==False]
tracks = df.drop(columns=["Type", "Race(s)", "Season(s)", "Races held"])
tracks

In [ ]:
#generating the country names from their codes using country_converter
tracks['Country Name'] = tracks.Country.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))
tracks

In [ ]:
#as the original codes were not in ISO3 format, some codes needed to be converted manually
df = tracks.replace({'Country Name':{'SIN': 'Singapore','MON': 'Monaco', 'UAE': 'United Arab Emirates', 'NED': 'Netherlands'}})
df

In [ ]:
#getting attendance data
data = pd.read_html('https://f1destinations.com/2017-f1-attendance-figures/')[0]
data = data[data["Race"].str.contains("TOTAL")==False]
data = data.drop(columns=["% Change"])

columns_titles = ["Race","2016 Attendance","2017 Attendance"]

data = data.reindex(columns=columns_titles)
data["Growth (%)"] = (((data["2017 Attendance"] / data["2016 Attendance"]) - 1) * 100).astype(int)
data['Race'] = data['Race'].str.replace('\d+', '', regex=True).str.replace('.', '', regex=True).str.replace(' ', '', regex=True)
data = data.replace({'Race':{'USA': 'United States', 'AbuDhabi': 'United Arab Emirates', 'GreatBritain': 'United Kingdom'}})
data

In [ ]:
join = pd.merge(df, data, left_on='Country Name', right_on='Race', how='left')
join

In [ ]:
geolocator = Nominatim(user_agent='test')
join['gcode'] = join['Location'].apply(geolocator.geocode)
join['lat'] = [g.latitude for g in join.gcode]
join['long'] = [g.longitude for g in join.gcode]
final = join.drop(columns=["gcode"])
final

In [ ]:
#building the map
def find_color(growth):
    if (growth > 20):
        return 'green'
    if (growth > 10):
        return 'yellow'
    else:
        return 'red'

map_clusters = folium.Map(zoom_start=4)

for lat, long, att, growth in zip(final['lat'], final['long'], final['2017 Attendance'], final['Growth (%)']):
    folium.CircleMarker(
    [lat, long],
    color='black',
    fill_color=find_color(growth),
    weight=1,
    fill_opacity = 0.6,
    radius = att/20000).add_to(map_clusters)
    
map_clusters

In [ ]:
from tkinter import *
from tkinter import filedialog
from tkinter import Tk
import json

# Quickly get rid of the root window popup
root = Tk()
root.withdraw()

# Use Filedialog.askopenfilename to open a dialog window where you can select your credentials file.
filepath = filedialog.askopenfilename()
file = open(filepath, 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

api_key = credentials['api_key']
client_id = credentials['client_id']
client_secret = credentials['client_secret']
category_id = '4eb1c1623b7b52c0e1adc2ec'
version = 1

print("api key:", api_key)
print("id:", client_id)
print("secret key:", client_secret)
